# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install dotenv

In [87]:
# from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [8]:
import pandas as pd
df = pd.read_csv(r"/content/Data.csv")

In [9]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [10]:
pip install langchain_openai

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [18]:
#Replace None by your own value and justify
llm = ChatOpenAI(
        openai_api_key=openai_api_key, # <--- Pass the key here!
        model="gpt-3.5-turbo",        # Specify your desired model
        temperature=0.7               # Set temperature
    )
print("ChatOpenAI model initialized successfully!")

ChatOpenAI model initialized successfully!


In [19]:
prompt = ChatPromptTemplate.from_template("Tell me about the product: {product_description}")


In [20]:
chain = LLMChain(llm=llm, prompt=prompt)


<ipython-input-20-1929250370>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [21]:
product = "Luxury Air Mattress"
chain.run(product)



<ipython-input-21-3125565976>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'A luxury air mattress is a high-quality, comfortable mattress that is designed to provide a luxurious sleeping experience. These mattresses are typically made with durable materials such as puncture-resistant PVC or heavy-duty vinyl, and are often constructed with multiple air chambers for customizable firmness levels.\n\nLuxury air mattresses often come with added features such as built-in pumps for easy inflation and deflation, plush pillow tops for extra comfort, and even adjustable settings for personalized sleep preferences. Some luxury air mattresses also have additional support layers, such as memory foam or gel-infused foam, to provide added comfort and pressure relief.\n\nThese mattresses are ideal for guests, camping trips, or as a temporary sleeping solution, as they are easy to set up and pack away when not in use. They are also a popular choice for individuals who want the convenience of an air mattress without sacrificing comfort and durability.\n\nOverall, a luxury air 

## SimpleSequentialChain

In [22]:
from langchain.chains import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(openai_api_key=openai_api_key,
        model="gpt-3.5-turbo", temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "As an expert in product marketing, analyze the following product description:\n\n{product_description}\n\n"
    "Based on this, suggest 3 concise, catchy, and benefit-oriented taglines. "
    "Format them as a numbered list."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [28]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on these taglines for a product:\n\n{taglines_from_first_chain}\n\n"
    "Suggest a creative, short social media post (e.g., for Instagram or X) "
    "that incorporates one of these taglines. Include relevant emojis and hashtags."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [29]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [30]:
product = "A revolutionary smart thermostat that learns your preferences, saves energy, and can be controlled remotely via a smartphone app. It features a sleek design and easy installation."
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
1. "Effortlessly save energy and stay comfortable with our intuitive smart thermostat!"
2. "Take control of your comfort and energy usage from anywhere with our innovative smart thermostat!"
3. "Sleek design, easy installation, and energy savings – all in one smart thermostat!"
"Stay cozy and save energy with our sleek smart thermostat! 🌡️💡 #SmartThermostat #EnergySavings #EffortlessComfort"

> Finished chain.


'"Stay cozy and save energy with our sleek smart thermostat! 🌡️💡 #SmartThermostat #EnergySavings #EffortlessComfort"'

**Repeat the above twice for different products**

In [35]:
overall_simple_chain.invoke(df["Review"][1])



> Entering new SimpleSequentialChain chain...
1. "Keep your valuables safe with our durable waterproof sac - worry-free protection!"
2. "Seamless waterproofing, but with easy access - never compromise on functionality!"
3. "Protect your devices on-the-go with our reliable waterproof sac - convenience at your fingertips!"
"Never fear the elements with our durable waterproof sac! 🌧️ Keep your valuables safe and secure no matter where your adventures take you. #WorryFreeProtection #WaterproofSac"

> Finished chain.


{'input': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'output': '"Never fear the elements with our durable waterproof sac! 🌧️ Keep your valuables safe and secure no matter where your adventures take you. #WorryFreeProtection #WaterproofSac"'}

In [34]:
overall_simple_chain(df["Review"][4])



> Entering new SimpleSequentialChain chain...
1. "Whip up your favorite coffee creations effortlessly with our durable frother!"
2. "Savor the perfect coffee every morning with our reliable frother that won't let you down."
3. "Elevate your coffee experience with a frother that lasts, so you can enjoy your daily brew worry-free."
"Start your day right with our durable frother ☕️ Say goodbye to clumpy coffee and hello to smooth, frothy perfection every time! #CoffeeLovers #MorningRoutine #Frother"

> Finished chain.


{'input': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'output': '"Start your day right with our durable frother ☕️ Say goodbye to clumpy coffee and hello to smooth, frothy perfection every time! #CoffeeLovers #MorningRoutine #Frother"'}

## SequentialChain

In [36]:
from langchain.chains import SequentialChain

In [73]:
llm = ChatOpenAI(openai_api_key=openai_api_key,
        model="gpt-3.5-turbo", temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate the following product review into {target_language}:\n\nReview: \"{review_text}\""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="english_review_output" # This matches 'english_review_output' in overall_chain's output_variables
                    )


In [53]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review in 1-2 concise sentences:\n\nReview: \"{review_text}\""
)
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="review_summary" # This matches 'review_summary' in overall_chain's output_variables
                    )

In [54]:
# prompt template 3: translate to english or other language

third_prompt = ChatPromptTemplate.from_template(
    "Translate the following summary into {target_language}:\n\nSummary: \"{review_summary}\"" # Uses output from chain_two
)
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_summary_output" # Fixed typo and named clearly
                      )

In [55]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Given the original product review:\n\"\"\"{review_text}\"\"\"\n\n" # Using {review_text} as the overall input
    "And its translated summary:\n\"\"\"{translated_summary_output}\"\"\"\n\n" # Using output from chain_three
    "Write a concise, empathetic, and professional follow-up message to the customer regarding their review. "
    "Offer a solution or next step if appropriate."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                       output_key="follow_up_message" # This matches 'follow_up_message' in overall_chain's output_variables
                      )

In [57]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review_text", "target_language"], # <-- This is the fix for your ValidationError!
    output_variables=["english_review_output", "review_summary", "follow_up_message"],
    verbose=True
)

In [67]:
review_text_to_process = df.Review[2]
target_language_for_translation = "Gujarati" # You specified Gujarati in your invoke call

# Correct way to invoke the chain:
# Pass a single dictionary with keys matching overall_chain's input_variables
results = overall_chain.invoke({
    "review_text": review_text_to_process,
    "target_language": target_language_for_translation
})

print("\n--- Processed Output (Line by Line) ---")
for key, value in results.items():
    print(f"{key.replace('_', ' ').title()}:") # Makes keys look nicer (e.g., "Review Text")
    print(f"  {value}\n") # Indent the value and add a blank line for separation



> Entering new SequentialChain chain...

> Finished chain.

--- Processed Output (Line by Line) ---
Review Text:
  This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.

Target Language:
  Gujarati

English Review Output:
  સમીક્ષા: "આ ગદિ પર થોડું છોટું છે (તેની સ્થાનપાણી મળવા માટે અનેક સમય લાગ્યું), અને તેમની પેચેઝ કામ નથી કર્યું, શાયદ કે તે ગદીનો પ્રથમ અંશ

In [68]:
review_text_to_process = df.Review[3]
target_language_for_translation = "Hindi" # You specified Gujarati in your invoke call

# Correct way to invoke the chain:
# Pass a single dictionary with keys matching overall_chain's input_variables
results = overall_chain.invoke({
    "review_text": review_text_to_process,
    "target_language": target_language_for_translation
})

print("\n--- Processed Output (Line by Line) ---")
for key, value in results.items():
    print(f"{key.replace('_', ' ').title()}:") # Makes keys look nicer (e.g., "Review Text")
    print(f"  {value}\n") # Indent the value and add a blank line for separation



> Entering new SequentialChain chain...

> Finished chain.

--- Processed Output (Line by Line) ---
Review Text:
  This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great

Target Language:
  Hindi

English Review Output:
  समीक्षा: "यह एमेज़ॅन पर सबसे अच्छे थ्रो पिलो फिलर्स हैं। मैंने कई अन्य को भी आज़माया है, और वे सभी सस्ते और समतल हैं चाहे आप उन्हें कितना भी फ्लफिंग करें। जब आप वैक्यूम सील्ड शिपिंग सामग्री से हटाकर उन्हें ड्रायर में फेंकते हैं, तो वे बहुत अच्छे फ़्लफ़ हो जाते हैं।"

Review Summary:
  These throw pillow fillers are the best on Amazon, providing excellent fluffiness when placed in the dryer after removing them from the vacuum sealed packaging.

Follow Up Message:
  Dear valued customer,

Thank you for taking the time to share your positive experience with our throw 

In [69]:
review_text_to_process = df.Review[6]
target_language_for_translation = "German" # You specified Gujarati in your invoke call

# Correct way to invoke the chain:
# Pass a single dictionary with keys matching overall_chain's input_variables
results = overall_chain.invoke({
    "review_text": review_text_to_process,
    "target_language": target_language_for_translation
})

print("\n--- Processed Output (Line by Line) ---")
for key, value in results.items():
    print(f"{key.replace('_', ' ').title()}:") # Makes keys look nicer (e.g., "Review Text")
    print(f"  {value}\n") # Indent the value and add a blank line for separation



> Entering new SequentialChain chain...

> Finished chain.

--- Processed Output (Line by Line) ---
Review Text:
  Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.

Target Language:
  German

English Review Output:
  Bewertung: "Es ist sehr hübsch, heizt sehr schnell auf, ist sehr funktional, muss nur noch sehen, wie lange es hält, ich benutze es erst seit 3 Tagen."

Review Summary:
  This product is very attractive and heats up quickly, it is functional and works well, but the reviewer can't comment on its durability as they have only been using it for 3 days.

Follow Up Message:
  Dear [Customer],

Thank you for your review of our product. We are thrilled to hear that you are pleased with its appearance, performance, and functionality. We understand your concern regarding its durability, given that you have only been using it for 3 days.

We are confident in the quality of our products and would like to assure yo

**Repeat the above twice for different products or reviews**

## Router Chain

In [70]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [71]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [72]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [75]:
llm = ChatOpenAI(openai_api_key=openai_api_key,
        model="gpt-3.5-turbo", temperature=0.6)

In [76]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [77]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [78]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [79]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [80]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-80-3038952769>:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [81]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of electromagnetic radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation in a continuous spectrum that depends only on its temperature. This radiation is described by Planck's law, which states that the intensity of radiation emitted by a black body is proportional to the fourth power of its temperature. This phenomenon is important in understanding concepts such as thermal radiation and the behavior of objects at high temperatures."

In [82]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [83]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, growth, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to maintain the overall functioning of the organism. Additionally, DNA replication ensures that genetic information is passed on to new cells during cell division, allowing for growth and repair of tissues.\n\nOverall, the presence of DNA in every cell is crucial for the proper functioning and survival of an organism.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [85]:
chain.run("Who was Gandhiji")



> Entering new MultiPromptChain chain...
History: {'input': 'Who was Gandhiji'}
> Finished chain.


'Gandhiji, also known as Mahatma Gandhi, was a prominent leader in the Indian independence movement against British rule. He was known for his philosophy of nonviolent resistance, which he called Satyagraha. Gandhi led numerous campaigns for civil rights, social justice, and independence, and his efforts ultimately played a significant role in India gaining independence from British rule in 1947. He is widely regarded as one of the most important figures in the history of India and a symbol of peace and nonviolence worldwide.'

In [86]:
chain.run("what precousion thyroid patient should take?")



> Entering new MultiPromptChain chain...
biology: {'input': 'what precautions should a thyroid patient take?'}
> Finished chain.


"As a biologist, I can provide some general guidelines for thyroid patients to consider:\n\n1. Take medication as prescribed: Thyroid patients typically need to take synthetic thyroid hormones to regulate their hormone levels. It's important to take the medication exactly as prescribed by your healthcare provider to ensure proper thyroid function.\n\n2. Get regular check-ups: Thyroid patients should have regular check-ups with their healthcare provider to monitor thyroid hormone levels and adjust medication dosage if necessary.\n\n3. Maintain a healthy diet: Eating a balanced diet with plenty of fruits, vegetables, whole grains, and lean proteins can help support thyroid function. Some thyroid patients may also benefit from avoiding certain foods that can interfere with thyroid hormone production, such as soy and cruciferous vegetables.\n\n4. Manage stress: Stress can affect thyroid function, so it's important for thyroid patients to find ways to manage stress through relaxation techni